In [63]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

In [64]:
data = pd.read_csv('./total_data.csv')
monthly = pd.read_csv('./month_stock.csv')
stocks = pd.read_csv('./trade_train_set/stocks.csv')
trade_train = pd.read_csv('./trade_train_set/trade_train.csv')

stocks = stocks.drop(['index'], axis=1)
trade_train = trade_train.drop(['Unnamed: 0'], axis=1)

In [65]:
len(data[data['20년7월TOP3대상여부']=='Y']['종목번호'].unique())

133

In [66]:
'''****** 변수 수정

거래량, 가격 -> 1을 기준으로 scaling, or 전월을 기준으로 +a%
20년도 7월에 포함되지 않은 것은 모두 drop


****** 확실히 버릴 것
그룹내_매수여부
그룹내_매도여부
기준년월
평균매수수량
평균매도수량
매수가격_중앙값
매도가격_중앙값
20년 7월 top3
종목명


******* 애매한 것
매도고객수 (전월 데이터를 사용할까?) -> 사용한다면 19년 7월은 어떻게 할까?

산업 대분류

산업 중분류

산업 소분류



******* 가져갈 것.

그룹번호
그룹내 고객수
종목번호
매수고객수 (Y)
거래량
거래금액_만원단위(정규화)
시장구분(정규화)
평균가격(정규화)
직전달 TOP3 여부'''

'****** 변수 수정\n\n거래량, 가격 -> 1을 기준으로 scaling, or 전월을 기준으로 +a%\n20년도 7월에 포함되지 않은 것은 모두 drop\n\n\n****** 확실히 버릴 것\n그룹내_매수여부\n그룹내_매도여부\n기준년월\n평균매수수량\n평균매도수량\n매수가격_중앙값\n매도가격_중앙값\n20년 7월 top3\n종목명\n\n\n******* 애매한 것\n매도고객수 (전월 데이터를 사용할까?) -> 사용한다면 19년 7월은 어떻게 할까?\n\n산업 대분류\n\n산업 중분류\n\n산업 소분류\n\n\n\n******* 가져갈 것.\n\n그룹번호\n그룹내 고객수\n종목번호\n매수고객수 (Y)\n거래량\n거래금액_만원단위(정규화)\n시장구분(정규화)\n평균가격(정규화)\n직전달 TOP3 여부'

In [67]:
data = data[data['20년7월TOP3대상여부'] == 'Y']

data = data.drop(['그룹내_매수여부', '그룹내_매도여부', '평균매수수량', '평균매도수량', 
                  '매수가격_중앙값', '매도가격_중앙값', '20년7월TOP3대상여부', '종목명'], axis=1)

In [68]:
a = trade_train[['기준년월', '그룹번호', '종목번호', '매수고객수']].groupby(['기준년월', '그룹번호',
                                                            '종목번호']).sum().sort_values(by = [
    '기준년월','그룹번호','매수고객수'], ascending=[True, True, False]).reset_index(drop=False)

a = a.groupby(['기준년월', '그룹번호']).head(3)

prev_month_list = list(a[a['기준년월'] == 202006]['종목번호'])

data['직전달TOP3여부'] = 0
for i in range(data.shape[0]):
    if data.iloc[i,3] in prev_month_list:
        data.iloc[i,-1] = 1

In [69]:
sum(data['직전달TOP3여부'] == 1)

4805

In [70]:
len(data[data['직전달TOP3여부'] == 1]['종목번호'].unique())

25

In [71]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
def dummy(data,col):
    lab=LabelEncoder()
    aa=lab.fit_transform(data[col]).reshape(-1,1)
    ohe=OneHotEncoder(sparse=False)
    column_names=[col+'_'+ str(i) for i in lab.classes_]
    return(pd.DataFrame(ohe.fit_transform(aa),columns=column_names))

In [72]:
categ_columns = ['그룹번호', '종목번호','표준산업구분코드_대분류', '표준산업구분코드_중분류', 
                 '표준산업구분코드_소분류', '시장구분']
n_columns = ['기준년월','그룹내고객수', '매수고객수', '거래량', '거래금액_만원단위', '평균가격', '직전달TOP3여부']

df = data[n_columns]
df.index = range(df.shape[0])

In [73]:
lab = LabelEncoder()
for col in categ_columns:
    temp_df = pd.DataFrame(lab.fit_transform(data[col]))
    temp_df.columns = [col]
    df = pd.concat([df, temp_df], axis=1)

In [74]:
df.to_csv('./df.csv',mode='w', header=True, index=False)

In [75]:
df

,기준년월,그룹내고객수,매수고객수,거래량,거래금액_만원단위,평균가격,직전달TOP3여부,그룹번호,종목번호,표준산업구분코드_대분류,표준산업구분코드_중분류,표준산업구분코드_소분류,시장구분
0,201907,288,7,2.993388e+06,2.244919e+07,74995.933792,1,0,3,9,26,19,1
1,201907,776,23,2.993388e+06,2.244919e+07,74995.933792,1,3,3,9,26,19,1
2,201907,890,3,2.993388e+06,2.244919e+07,74995.933792,1,4,3,9,26,19,1
3,201907,1615,27,2.993388e+06,2.244919e+07,74995.933792,1,5,3,9,26,19,1
4,201907,634,7,2.993388e+06,2.244919e+07,74995.933792,1,8,3,9,26,19,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7485,202006,2879,6,1.061898e+05,9.436800e+05,88867.277123,0,42,26,4,10,22,1
7486,202006,2879,0,1.635518e+06,6.335980e+05,3873.990846,0,42,32,9,14,12,1
7487,202006,2879,3,5.442303e+05,3.067193e+05,5635.835831,0,42,58,9,7,50,0
7488,202006,2879,6,1.758884e+05,5.464499e+05,31067.988614,0,42,100,8,29,26,0


In [76]:
df_temp = pd.DataFrame()

기준년월 = pd.Series(df['기준년월'].unique())
그룹번호 = pd.Series(df['그룹번호'].unique())
종목번호 = pd.Series(df['종목번호'].unique())

aa = []
for k in 기준년월:
    for i in 그룹번호:
        for j in 종목번호:
            aa.append([k,i,j])
        
aa = pd.DataFrame(data=aa, columns=['기준년월','그룹번호', '종목번호'])        
df_temp[['기준년월', '그룹번호', '종목번호']] = aa[['기준년월', '그룹번호', '종목번호']]

In [77]:
df_temp = pd.merge(df_temp, df[['종목번호', '직전달TOP3여부', '표준산업구분코드_대분류', '표준산업구분코드_중분류', 
                 '표준산업구분코드_소분류', '시장구분']].drop_duplicates(), on='종목번호')

df_temp = pd.merge(df_temp, df[['그룹번호','그룹내고객수']].drop_duplicates(), on='그룹번호')

df_temp = pd.merge(df_temp, df[['매수고객수','기준년월', '그룹번호', '종목번호', '거래량', 
                                '거래금액_만원단위', '평균가격']], 
        on = ['기준년월', '그룹번호', '종목번호'], how='outer')

df_temp = df_temp.fillna(0)


In [78]:
df_temp = df_temp[['기준년월', '그룹내고객수', '매수고객수', '거래량', '거래금액_만원단위', '평균가격', 
                   '직전달TOP3여부', '그룹번호', '종목번호', '표준산업구분코드_대분류', '표준산업구분코드_중분류', 
                   '표준산업구분코드_소분류', '시장구분']]

In [79]:
df_temp

,기준년월,그룹내고객수,매수고객수,거래량,거래금액_만원단위,평균가격,직전달TOP3여부,그룹번호,종목번호,표준산업구분코드_대분류,표준산업구분코드_중분류,표준산업구분코드_소분류,시장구분
0,201907,288,7.0,2.993388e+06,2.244919e+07,74995.933792,1,0,3,9,26,19,1
1,201908,288,17.0,2.506382e+06,1.872768e+07,74719.983788,1,0,3,9,26,19,1
2,201909,288,12.0,2.542724e+06,2.069622e+07,81393.895693,1,0,3,9,26,19,1
3,201910,288,17.0,3.030296e+06,2.418127e+07,79798.364529,1,0,3,9,26,19,1
4,201911,288,18.0,2.173125e+06,1.806452e+07,83126.886985,1,0,3,9,26,19,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
76603,202002,86,0.0,0.000000e+00,0.000000e+00,0.000000,0,6,132,9,14,17,0
76604,202003,86,0.0,0.000000e+00,0.000000e+00,0.000000,0,6,132,9,14,17,0
76605,202004,86,0.0,0.000000e+00,0.000000e+00,0.000000,0,6,132,9,14,17,0
76606,202005,86,0.0,0.000000e+00,0.000000e+00,0.000000,0,6,132,9,14,17,0


In [60]:
dat_mean = df_temp[['거래금액_만원단위', '평균가격', '거래량']].mean(axis=0)

dat_std = df_temp[['거래금액_만원단위', '평균가격', '거래량']].std()

from sklearn.preprocessing import StandardScaler
df_temp[['거래금액_만원단위', '평균가격', '거래량']] = StandardScaler(
).fit_transform(df_temp[['거래금액_만원단위', '평균가격', '거래량']])

df_temp

,기준년월,그룹내고객수,매수고객수,거래량,거래금액_만원단위,평균가격,직전달TOP3여부,그룹번호,종목번호,표준산업구분코드_대분류,표준산업구분코드_중분류,표준산업구분코드_소분류,시장구분
0,201907,288,7.0,0.939327,2.198725,1.531929,1,0,3,9,26,19,1
1,201908,288,17.0,0.759884,1.806019,1.525568,1,0,3,9,26,19,1
2,201909,288,12.0,0.773275,2.013745,1.679414,1,0,3,9,26,19,1
3,201910,288,17.0,0.952926,2.381499,1.642634,1,0,3,9,26,19,1
4,201911,288,18.0,0.637092,1.736039,1.719363,1,0,3,9,26,19,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
76603,202002,86,0.0,-0.163618,-0.170185,-0.196868,0,6,132,9,14,17,0
76604,202003,86,0.0,-0.163618,-0.170185,-0.196868,0,6,132,9,14,17,0
76605,202004,86,0.0,-0.163618,-0.170185,-0.196868,0,6,132,9,14,17,0
76606,202005,86,0.0,-0.163618,-0.170185,-0.196868,0,6,132,9,14,17,0


In [61]:
df_temp.to_csv('./df_temp.csv',mode='w', header=True, index=False)

In [62]:
df_temp

,기준년월,그룹내고객수,매수고객수,거래량,거래금액_만원단위,평균가격,직전달TOP3여부,그룹번호,종목번호,표준산업구분코드_대분류,표준산업구분코드_중분류,표준산업구분코드_소분류,시장구분
0,201907,288,7.0,0.939327,2.198725,1.531929,1,0,3,9,26,19,1
1,201908,288,17.0,0.759884,1.806019,1.525568,1,0,3,9,26,19,1
2,201909,288,12.0,0.773275,2.013745,1.679414,1,0,3,9,26,19,1
3,201910,288,17.0,0.952926,2.381499,1.642634,1,0,3,9,26,19,1
4,201911,288,18.0,0.637092,1.736039,1.719363,1,0,3,9,26,19,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
76603,202002,86,0.0,-0.163618,-0.170185,-0.196868,0,6,132,9,14,17,0
76604,202003,86,0.0,-0.163618,-0.170185,-0.196868,0,6,132,9,14,17,0
76605,202004,86,0.0,-0.163618,-0.170185,-0.196868,0,6,132,9,14,17,0
76606,202005,86,0.0,-0.163618,-0.170185,-0.196868,0,6,132,9,14,17,0


In [20]:
# 예측 템플릿 만들기
그룹번호 = pd.Series(df['그룹번호'].unique())
종목번호 = pd.Series(df['종목번호'].unique())


temp = []
for i in 그룹번호:
    for j in 종목번호:
        temp.append([i, j])

                                
temp = pd.DataFrame(data=temp, columns = ['그룹번호', '종목번호'])

In [21]:
a = trade_train[['그룹번호','그룹내고객수']].groupby('그룹번호').first().reset_index(drop=False)
a['그룹번호'] = lab.fit_transform(a['그룹번호'])

temp = pd.merge(temp, a, on='그룹번호')

temp = pd.merge(temp, df[['표준산업구분코드_대분류','표준산업구분코드_중분류', '표준산업구분코드_소분류', 
                          '종목번호', '직전달TOP3여부', '시장구분']].drop_duplicates(), on='종목번호')

In [22]:
a = stocks[stocks['기준일자'] > 20200700].groupby(['종목번호']).sum().reset_index(drop=False)[[
    '종목번호', '거래금액_만원단위', '거래량']]
a['종목번호'] = lab.fit_transform(a['종목번호'])

a['종목번호'] = lab.fit_transform(a['종목번호'])

a['거래금액_만원단위'] = a['거래금액_만원단위']
a['평균가격'] = (a['거래금액_만원단위'] / a['거래량']) * 10000


In [23]:
temp = pd.merge(temp, a, on='종목번호')

temp['거래금액_만원단위'] = (temp['거래금액_만원단위'] - dat_mean[0]) / dat_std[0]
temp['평균가격'] = (temp['평균가격'] - dat_mean[1]) / dat_std[1]
temp['거래량'] = (temp['거래량'] - dat_mean[2]) / dat_std[2]

In [24]:
temp = temp[['그룹내고객수','거래량','거래금액_만원단위','평균가격','직전달TOP3여부', '그룹번호', 
      '종목번호','표준산업구분코드_대분류', '표준산업구분코드_중분류', '표준산업구분코드_소분류', '시장구분']]

temp.to_csv('./template.csv',mode='w', header=True, index=False)

In [25]:
temp

,그룹내고객수,거래량,거래금액_만원단위,평균가격,직전달TOP3여부,그룹번호,종목번호,표준산업구분코드_대분류,표준산업구분코드_중분류,표준산업구분코드_소분류,시장구분
0,288,0.203312,1.503118,3.473745,1,0,3,9,26,19,1
1,776,0.203312,1.503118,3.473745,1,3,3,9,26,19,1
2,890,0.203312,1.503118,3.473745,1,4,3,9,26,19,1
3,1615,0.203312,1.503118,3.473745,1,5,3,9,26,19,1
4,634,0.203312,1.503118,3.473745,1,8,3,9,26,19,1
...,...,...,...,...,...,...,...,...,...,...,...
6379,115,6.878505,0.202351,-0.154286,0,28,132,9,14,17,0
6380,204,6.878505,0.202351,-0.154286,0,44,132,9,14,17,0
6381,106,6.878505,0.202351,-0.154286,0,9,132,9,14,17,0
6382,82,6.878505,0.202351,-0.154286,0,39,132,9,14,17,0


In [26]:
df_temp.columns = ['date', 'custom_in_grp', 'buyers', 'volume', 'trade_total', 'avg_price', 
                   'top3', 'grp', 'stock', 'big', 'mid', 'small', 'market']

df_temp

,date,custom_in_grp,buyers,volume,trade_total,avg_price,top3,grp,stock,big,mid,small,market
0,201907,288,7.0,0.939327,2.198725,1.531929,1,0,3,9,26,19,1
1,201908,288,17.0,0.759884,1.806019,1.525568,1,0,3,9,26,19,1
2,201909,288,12.0,0.773275,2.013745,1.679414,1,0,3,9,26,19,1
3,201910,288,17.0,0.952926,2.381499,1.642634,1,0,3,9,26,19,1
4,201911,288,18.0,0.637092,1.736039,1.719363,1,0,3,9,26,19,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
76603,202002,86,0.0,-0.163618,-0.170185,-0.196868,0,6,132,9,14,17,0
76604,202003,86,0.0,-0.163618,-0.170185,-0.196868,0,6,132,9,14,17,0
76605,202004,86,0.0,-0.163618,-0.170185,-0.196868,0,6,132,9,14,17,0
76606,202005,86,0.0,-0.163618,-0.170185,-0.196868,0,6,132,9,14,17,0


In [27]:
import lightgbm as lgb

train = df_temp[df_temp['date'] != 202006]
val = df_temp[df_temp['date'] == 202006]

/opt/anaconda3/envs/venv/lib/python3.8/site-packages/lightgbm/__init__.py:42: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  warnings.warn("Starting from version 2.2.1, the library file in distribution wheels for macOS "


In [28]:
x_train = train.drop(['date','buyers'], axis=1)
y_train = train['buyers']
x_val = val.drop(['date','buyers'], axis=1)
y_val = val['buyers']

x_train

,custom_in_grp,volume,trade_total,avg_price,top3,grp,stock,big,mid,small,market
0,288,0.939327,2.198725,1.531929,1,0,3,9,26,19,1
1,288,0.759884,1.806019,1.525568,1,0,3,9,26,19,1
2,288,0.773275,2.013745,1.679414,1,0,3,9,26,19,1
3,288,0.952926,2.381499,1.642634,1,0,3,9,26,19,1
4,288,0.637092,1.736039,1.719363,1,0,3,9,26,19,1
...,...,...,...,...,...,...,...,...,...,...,...
76602,86,-0.163618,-0.170185,-0.196868,0,6,132,9,14,17,0
76603,86,-0.163618,-0.170185,-0.196868,0,6,132,9,14,17,0
76604,86,-0.163618,-0.170185,-0.196868,0,6,132,9,14,17,0
76605,86,-0.163618,-0.170185,-0.196868,0,6,132,9,14,17,0


In [29]:
categ=['grp', 'big', 'stock', 'mid', 'small']
train_ds = lgb.Dataset(x_train, label=y_train, categorical_feature=categ, free_raw_data=False)
val_ds = lgb.Dataset(x_val, label=y_val, categorical_feature=categ, free_raw_data=False)

In [30]:
def lgbm_regressor(train_X, train_y, test_X, test_y, feature_names=None, 
                   seed_val=2284, num_rounds=5000):

    param = {}

    param['boosting'] = 'gbdt'
    
    param['objective'] = 'regression'

    param['learning_rate'] = 0.05

    param['max_depth'] = 10

    param['metric'] = 'rmse'
    
    param['min_data_in_leaf'] = 100
    
    param['is_training_metric'] = True
    
    param['bagging_fraction'] = 0.8
    
    param['num_leaves'] = 1000

    param['feature_fraction'] = 0.8
    
    param['seed'] = seed_val
    
    num_rounds = num_rounds

    plst = list(param.items())

    model = lgb.train(param, train_ds, num_rounds,val_ds, verbose_eval= 100, 
                      early_stopping_rounds=180)

    return model

model = lgbm_regressor(train_X = x_train, train_y = y_train, test_X = x_val, test_y = y_val)

/opt/anaconda3/envs/venv/lib/python3.8/site-packages/lightgbm/basic.py:1243: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 180 rounds
[100]	valid_0's rmse: 13.5359
[200]	valid_0's rmse: 12.1915
[300]	valid_0's rmse: 11.4462
[400]	valid_0's rmse: 10.9189
[500]	valid_0's rmse: 10.5698
[600]	valid_0's rmse: 10.2554
[700]	valid_0's rmse: 9.99982
[800]	valid_0's rmse: 9.83821
[900]	valid_0's rmse: 9.69148
[1000]	valid_0's rmse: 9.55026
[1100]	valid_0's rmse: 9.44999
[1200]	valid_0's rmse: 9.36768
[1300]	valid_0's rmse: 9.30459
[1400]	valid_0's rmse: 9.24279
[1500]	valid_0's rmse: 9.18108
[1600]	valid_0's rmse: 9.14398
[1700]	valid_0's rmse: 9.10371
[1800]	valid_0's rmse: 9.04935
[1900]	valid_0's rmse: 8.99184
[2000]	valid_0's rmse: 8.95604
[2100]	valid_0's rmse: 8.91026
[2200]	valid_0's rmse: 8.85323
[2300]	valid_0's rmse: 8.81905
[2400]	valid_0's rmse: 8.77426
[2500]	valid_0's rmse: 8.7281
[2600]	valid_0's rmse: 8.68622
[2700]	valid_0's rmse: 8.64805
[2800]	valid_0's rmse: 8.60996
[2900]	valid_0's rmse: 8.57204
[3000]	valid_0's rmse: 8.54514
[3100]	valid_0's r

In [31]:
temp.columns = x_train.columns

pred = model.predict(temp)

pred

array([ 9.29172285, 10.03062062,  9.59441614, ...,  0.69855275,
       -0.03431292,  0.42716641])

In [32]:
a = pd.DataFrame(pred, columns=['buyers'])

result = pd.concat([temp, a], axis = 1)

result

,custom_in_grp,volume,trade_total,avg_price,top3,grp,stock,big,mid,small,market,buyers
0,288,0.203312,1.503118,3.473745,1,0,3,9,26,19,1,9.291723
1,776,0.203312,1.503118,3.473745,1,3,3,9,26,19,1,10.030621
2,890,0.203312,1.503118,3.473745,1,4,3,9,26,19,1,9.594416
3,1615,0.203312,1.503118,3.473745,1,5,3,9,26,19,1,41.625433
4,634,0.203312,1.503118,3.473745,1,8,3,9,26,19,1,8.039113
...,...,...,...,...,...,...,...,...,...,...,...,...
6379,115,6.878505,0.202351,-0.154286,0,28,132,9,14,17,0,0.397283
6380,204,6.878505,0.202351,-0.154286,0,44,132,9,14,17,0,1.815703
6381,106,6.878505,0.202351,-0.154286,0,9,132,9,14,17,0,0.698553
6382,82,6.878505,0.202351,-0.154286,0,39,132,9,14,17,0,-0.034313


In [33]:
tmp = result[['grp', 'stock', 'buyers']].sort_values(by = ['grp','buyers'], 
                                                     ascending=[True, False])

tmp  = tmp.groupby(['grp']).head(3)
tmp[:-100]

,grp,stock,buyers
2784,0,121,150.945011
3936,0,115,150.879356
1152,0,31,144.359331
2817,1,121,51.243715
3969,1,115,51.203694
1185,1,31,46.673060
2819,2,121,68.493195
3971,2,115,68.479770
4547,2,70,64.820224
2785,3,121,226.069832


In [34]:
lab1 = LabelEncoder()
lab1.fit(data['그룹번호'])
lab2 = LabelEncoder()
lab2.fit(data['종목번호'])

LabelEncoder()

In [84]:
print(lab2.inverse_transform([121, 115, 70]))
print(lab1.inverse_transform([47]))

['A257370' 'A234100' 'A052260']
['MAD48']


In [117]:
df_dict = pd.DataFrame()
df_dict['종목번호'] = temp['stock']
df_dict['종목번호_원본'] = 0
df_dict
for i in range(df_dict.shape[0]):
    df_dict['종목번호_원본'][i] =  lab2.inverse_transform([df_dict['종목번호'][i]])[0]
        

<ipython-input-117-a5f27f41ae3c>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dict['종목번호_원본'][i] =  lab2.inverse_transform([df_dict['종목번호'][i]])[0]


In [123]:
df_dict = df_dict.drop_duplicates()

df_dict = df_dict.sort_values(by = ['종목번호'], ascending=[True])
df_dict.index = range(df_dict.shape[0])

df_dict.to_csv('./종목변환.csv',mode='w', header=True, index=False)

In [125]:
df_dict2 = pd.DataFrame()
df_dict2['그룹번호'] = temp['grp']
df_dict2['그룹번호_원본'] = 0

for i in range(df_dict2.shape[0]):
    df_dict2['그룹번호_원본'][i] =  lab1.inverse_transform([df_dict2['그룹번호'][i]])[0]

<ipython-input-125-774ad86bc7ea>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dict2['그룹번호_원본'][i] =  lab1.inverse_transform([df_dict2['그룹번호'][i]])[0]


In [127]:
df_dict2 = df_dict2.drop_duplicates()

df_dict2 = df_dict2.sort_values(by = ['그룹번호'], ascending=[True])
df_dict2.index = range(df_dict2.shape[0])

df_dict2.to_csv('./그룹변환.csv',mode='w', header=True, index=False)